In [1]:
%cd ..

C:\Users\13661\PycharmProjects\KinshipVerification


In [2]:
import numpy as np
import torch
import torchvision
from torch.utils.data.dataloader import DataLoader

device = torch.device('cuda')

In [3]:
from dataset import Triplet

data_path = 'D:/文档/硕士/Thesis/UvA-NEMO_SMILE_DATABASE/aligned'

In [4]:
train_split = np.load('train.npy')
dataset = Triplet(data_path, train_split)
loader = DataLoader(
        dataset, batch_size=1, shuffle=False, num_workers=1)

In [5]:
from transformers import ViTFeatureExtractor, ViTModel
from datasets import load_dataset

feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")

vit = ViTModel.from_pretrained("google/vit-base-patch16-224-in21k").eval().to(device)


In [6]:
from tqdm import tqdm
import torch.nn.functional as F

outs = []
with torch.no_grad():
    with tqdm(loader, unit="batch") as data_loader:
        for batch in data_loader:
            batch = [feature_extractor(list(v.squeeze().permute(0,3,1,2)), return_tensors="pt").to(device) for v in batch]

            anchor = vit(**batch[0])[0][:, 0].detach().mean(0).cpu()
            pos = vit(**batch[1])[0][:, 0].detach().mean(0).cpu()
            neg = vit(**batch[2])[0][:, 0].detach().mean(0).cpu()
            
            outs.append([F.normalize(torch.abs(anchor - pos), dim=0).numpy(), 1])
            outs.append([F.normalize(torch.abs(anchor - neg), dim=0).numpy(), 0])

100%|███████████████████████████████████████████████████████████████████████████████| 52/52 [02:54<00:00,  3.35s/batch]


In [7]:
import random
random.shuffle(outs)

X = [a[0] for a in outs]
Y = [a[1] for a in outs]

In [8]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=2, shuffle=True)

In [9]:
from sklearn import svm
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn.metrics import recall_score

clf = svm.SVC()

# scoring = {'prec_macro': 'precision_macro','rec_macro': make_scorer(recall_score, average='macro')}
# scores = cross_validate(clf,X, Y,
#                        scoring=scoring, cv=kf,
#                        return_estimator=True)
clf.fit(X,Y)
clf.score(X,Y)
# print(scores)

0.9519230769230769

In [10]:
# print(scores['test_prec_macro'])

In [11]:
test_split = np.load('test.npy')
testset = Triplet(data_path, test_split)
testloader = DataLoader(
        testset, batch_size=1, shuffle=False, num_workers=1)

In [12]:
from tqdm import tqdm
import torch.nn.functional as F

t_outs = []
with torch.no_grad():
    with tqdm(testloader, unit="batch") as data_loader:
        for batch in data_loader:
            batch = [feature_extractor(list(v.squeeze().permute(0,3,1,2)), return_tensors="pt").to(device) for v in batch]

            anchor = vit(**batch[0])[0][:, 0].detach().mean(0).cpu()
            pos = vit(**batch[1])[0][:, 0].detach().mean(0).cpu()
            neg = vit(**batch[2])[0][:, 0].detach().mean(0).cpu()
                        
            t_outs.append([F.normalize(torch.abs(anchor - pos), dim=0).numpy(), 1])
            t_outs.append([F.normalize(torch.abs(anchor - neg), dim=0).numpy(), 0])

100%|███████████████████████████████████████████████████████████████████████████████| 14/14 [00:48<00:00,  3.49s/batch]


In [13]:
random.shuffle(t_outs)

test_x = [a[0] for a in t_outs]
test_y = [a[1] for a in t_outs]

In [14]:
clf.score(test_x, test_y)

0.42857142857142855